# Анализ и обработка отзывов

In [ ]:
import pandas as pd
from tqdm import tqdm
import pymorphy3

# Инициализация морфологического анализатора для работы с русским языком
morph_analyzer = pymorphy3.MorphAnalyzer()


class CreateDataset:
	def __init__(self):
		# Определяем ключевые слова для категорий отзывов
		self.practice = {'задачи', 'кейс', 'проект', 'задания'}  # Слова, связанные с практикой
		self.theory = {'знания', 'лекция', 'материал'}  # Слова, связанные с теорией
		self.technology = {'проблема', 'ресурсы', 'инструменты', 'доступность',
						   'интерактивность'}  # Слова, связанные с технологиями
		self.teacher = {'лектор', 'учитель', 'профессор', 'наставник', 'педагог', 'препод', 'автор',
						'создатель'}  # Слова, связанные с преподавателем
		self.relevance = {'полезный', 'современный', 'релевантный', 'новый', 'старый',
						  'устарел'}  # Слова, связанные с актуальностью

		# Список ключевых тем
		self.topic_keywords = ['практика', 'теория', 'преподаватель', 'технологии', 'актуальность']

		# Создаем словарь, где ключевые темы связываются с набором соответствующих слов
		self.topics = {keyword: {keyword} for keyword in self.topic_keywords}

		# Обновляем словарь ключевых тем с учетом различных словоформ
		self.topics['практика'].update(self.practice)
		self.topics['теория'].update(self.theory)
		self.topics['преподаватель'].update(self.teacher)
		self.topics['технологии'].update(self.technology)
		self.topics['актуальность'].update(self.relevance)

	def names_in_text(self, text: str) -> bool:
		"""
        Проверяет, содержатся ли в тексте имена преподавателей.
        :param text: Исходный текст
        :return: True, если найдено хотя бы одно имя, иначе False
        """
		stop_names = frozenset('паскаль')  # Исключаемое имя (например, Паскаль)
		for word in text:
			if word not in stop_names:
				parse = morph_analyzer.parse(word)[0]
				tag = parse.tag
				# Проверяем, является ли слово именем или фамилией с высокой вероятностью
				if ('Name' in tag or 'Surn' in tag) and parse.score >= 0.8:
					return True
		return False

	def automatic_annotation(self, df: pd.DataFrame) -> pd.DataFrame:
		"""
        Автоматическая разметка обучающего набора данных на основе ключевых слов и имен преподавателей.
        :param df: DataFrame с текстами для разметки
        :return: Размеченный DataFrame
        """
		# Подготовка структуры для хранения данных
		data_all = {t: [] for t in self.topic_keywords}
		data_index = set()

		# Проходим по каждому тексту в DataFrame
		for idx, row in tqdm(df.iterrows(), total=len(df)):
			preprocessed_text = row['PreprocessedText']  # Предобработанный текст
			data = []
			for key, keywords in self.topics.items():
				# Проверяем наличие ключевых слов в тексте с использованием нечеткого поиска
				if any(True for keyword in keywords for word in preprocessed_text if fuzz.ratio(word, keyword) > 78):
					data_index.add(idx)
					data.append(1)
				elif key == 'преподаватель' and self.names_in_text(preprocessed_text):
					# Дополнительная проверка на наличие имен преподавателей
					data_index.add(idx)
					data.append(1)
				else:
					# Если текст не относится к категории
					data.append(0)
			if sum(data) > 0:
				# Добавляем данные в соответствующие столбцы
				for i, key in enumerate(self.topic_keywords):
					data_all[key].append(data[i])
		return pd.DataFrame(data_all, index=sorted(data_index))

## Обработка текста

In [ ]:
# Импортируем датасет с отзывами
import pandas as pd
from thefuzz import fuzz  # Новая версия библиотеки fuzzywuzzy
import nltk

# Загрузка необходОгромная благодарность авторам за труд! Полезный материал пра…имых данных для NLTK
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# from utils import CreateDataset

# Загрузка стоп-слов
stop_words = stopwords.words('russian')

# Загрузка данных
df = pd.read_csv('../data/train_reviews.csv', index_col=0)
df['Reviews'] = df['Reviews'].astype(str)

# Замена кириллической 'с' на латинскую
df['Reviews'] = df['Reviews'].apply(
	lambda x: x.replace('1с', '1c').replace('с#', 'c#').replace('с+', 'c+').replace('Ё', 'Е').replace(
		'1С', '1C').replace('С#', 'C#').replace('С+', 'C+'))


class TextProcessor:
	def preprocess(self, text: str) -> list[str]:
		"""
        Предобрабатывает текст, токенизируя, лемматизируя и удаляя стоп-слова.

        :param text: Входной текст, который необходимо предобработать.
        :return: Список предобработанных слов.
        """
		tokens = word_tokenize(text.lower())
		filtered_tokens = [
			normalized_word
			for word in tokens
			if (normalized_word := morph_analyzer.parse(self._remove_symbols(word))[0].normal_form) not in stop_words
		]

		filtered_tokens = " ".join(filtered_tokens).replace('ё', 'е').split()  # Удаление лишних пробелов
		return filtered_tokens

	@staticmethod
	def _remove_symbols(text: str) -> str:
		"""Удаляет все символы и цифры из строки."""
		clean_text = "".join(char if char.isalnum() and not char.isdigit() else " " for char in text)
		return clean_text

In [ ]:
text_processor = TextProcessor()

# Применение предобработки к отзывам
df['PreprocessedText'] = df['Reviews'].apply(text_processor.preprocess)

# Создание списка всех стеммированных слов
all_words = [word for sublist in df['PreprocessedText'] for word in sublist]
unique_words = pd.Series(all_words).value_counts()

# Создание DataFrame с размеченными данными
create_dataset = CreateDataset()
df_dataset_index = create_dataset.automatic_annotation(df)

Просмотр размеченных текстов

In [ ]:
import matplotlib.pyplot as plt

print('Всего размеченных текстов -', len(df_dataset_index))
for t in create_dataset.topic_keywords:
	print(t, '-', len(df_dataset_index[df_dataset_index[t] == 1]))

# Создание списков для тем и их количества
topics = create_dataset.topic_keywords
counts = [len(df_dataset_index[df_dataset_index[t] == 1]) for t in topics]

# Визуализация
plt.figure(figsize=(10, 6))
plt.bar(topics, counts, color='skyblue')
plt.xlabel('Темы')
plt.ylabel('Количество размеченных текстов')
plt.title('Количество размеченных текстов по темам')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


Размеченный и обработанный датасет

In [ ]:
# Размечиваем и обрабатываем датасет
df_marked: pd.DataFrame = df.loc[df_dataset_index.index]
for topic in create_dataset.topic_keywords:
	df_marked[topic] = df_dataset_index[topic]
df_marked['PreprocessedText'] = df_marked['PreprocessedText'].apply(lambda x: " ".join(x))

# Векторизация текста и обучение модели

## Векторизация

In [ ]:
# Размечиваем и обрабатываем датасет
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score

# Векторизация текста
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(df_marked['PreprocessedText'])
y = df_marked.drop(['PreprocessedText', 'Reviews'], axis=1)
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, df_marked[create_dataset.topic_keywords], test_size=0.3,
													random_state=42)


## Обучение модели

In [ ]:
# Обучаем модель классификации отзывов
# Используем многоцелевой классификатор
model = MultiOutputClassifier(DecisionTreeClassifier())
model.fit(X_train, y_train)

# Прогнозируем на тестовых данных
y_pred = model.predict(X_test)
# Оценка модели
print(classification_report(y_test, y_pred, target_names=y.columns, zero_division=0))
print(model.score(X_test, y_test))
print(f1_score(y_test, y_pred, average='macro'))

### Получим важность признаков


In [ ]:
import numpy as np

for i, name in enumerate(list(y.columns)):
	importances = model.estimators_[i].feature_importances_
	
	# Отсортируем важность
	indices = np.argsort(importances)[::-1]
	
	# Напечатаем топ важнейших признаков
	feature_names = vectorizer.get_feature_names_out()
	top_n = 7
	print('-'*32)
	print(f"Топ {top_n} важнейших слов для категории '{name}':")
	for i in range(top_n):
		if importances[indices[i]]:
			print(f"{feature_names[indices[i]]}: {importances[indices[i]]}")


### Проверка тестового текста

In [ ]:
test_review = "Крутой курс! Мне понравились задачи и препод"
test_review = (text_processor.preprocess(test_review))
test_review = " ".join(test_review)
new_vector = vectorizer.transform(pd.Series(test_review))
test_pred = model.predict(new_vector)
for i, k in enumerate(create_dataset.topic_keywords):
	print(k, ':', test_pred[0][i])

# Создание ответов

## Cоздание ответов с помощью модели

In [ ]:
df_test = pd.read_csv('../data/test_reviews.csv', index_col=0)
df_test['Reviews'] = df_test['Reviews'].astype(str)
df_test['Pred'] = df_test['Reviews'].apply(lambda x: " ".join(text_processor.preprocess(x)))
test_vector = vectorizer.transform(df_test['Pred'])

In [ ]:
test_predict = pd.DataFrame(model.predict(test_vector))

In [ ]:
for i, k in enumerate(create_dataset.topic_keywords):
	df_test[k] = test_predict[i]
df_pred = df_test.drop('Pred', axis=1)
df_pred.to_csv('../data/model_answer.csv', index=True)

## Разметка при помощи алгоритма

Добавим 'пустую' разметку

In [ ]:
# Пустые классы делают обучение bert более качественным
void_df = df_test[(df_test['Pred'].str.split().str.len() <= 5) &
				 (df_test['практика'] == 0) &
				 (df_test['теория'] == 0) &
				 (df_test['преподаватель'] == 0) &
				 (df_test['технологии'] == 0) &
				 (df_test['актуальность'] == 0)]
void_df = void_df[:50]
void_df['PreprocessedText'] = void_df['Pred']
void_df = void_df.drop('Pred', axis=1)

In [ ]:
# Размечиваем и рандомно перемешиваем датасет
df_auto = pd.concat([void_df.drop('PreprocessedText', axis=1), df_marked.drop('PreprocessedText', axis=1)], axis=0,
				   ignore_index=True)
df_auto = df_auto.sample(frac=1).reset_index(drop=True)
df_auto.to_csv('../data/train_reviews_bert.csv', index=False) # Можно использовать для обучения bert

## Комбинируем предсказания модели и автоматической разметки (логическое 'или')

In [ ]:
# Предсказываем категории отзывов на основе обученной модели
df_comb = df_pred.copy()
df_comb['практика']  |= df_test['практика']
df_comb['теория'] |= df_test['теория']
df_comb['преподаватель'] |= df_test['преподаватель']
df_comb['технологии'] |= df_test['технологии']
df_comb['актуальность'] |= df_test['актуальность']

df_comb.to_csv('../data/combined_answer.csv')